In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt

2024-04-14 21:28:49.657018: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
train_file = "enemyHead_train.tfrecord"
train_dataset = tf.data.TFRecordDataset(train_file)

valid_file = "enemyHead_valid.tfrecord"
valid_dataset = tf.data.TFRecordDataset(valid_file)

In [4]:
# for raw_record in dataset.take(10):
#     print(repr(raw_record))
    

In [21]:
from functools import partial

keys_to_features = {
    'image/encoded': tf.io.FixedLenFeature([], tf.string),
    'image/object/class/text': tf.io.VarLenFeature(tf.string),
    'image/object/class/label': tf.io.VarLenFeature(tf.int64),
}

def _parse_function(proto, returnText=False):
    # Load one example
    parsed_features = tf.io.parse_single_example(proto, keys_to_features)
    
    # Decode the image
    image = tf.image.decode_jpeg(parsed_features['image/encoded'], channels=3)
    image = tf.image.resize(image, [416, 416])
    image = image / 255.0  # Normalize pixel values
    
    # Convert sparse tensor to dense, handling multiple entries if necessary
    label = tf.sparse.to_dense(parsed_features['image/object/class/label'])
    text = tf.sparse.to_dense(parsed_features['image/object/class/text'])

    # Decode bytes to string for text
    text = tf.cast(text, tf.string)
    
    # Check if the label contains '2', and set label to True in that case
    contains_label_2 = tf.reduce_any(tf.equal(label, 2))
    label = tf.cast(contains_label_2, tf.float32)  # Convert boolean to float (1.0 for True, 0.0 for False)

    if returnText:
        return image, label, text
    return image, label

def save_images(dataset):
    parseFunction = partial(_parse_function, returnText=True)
    # Map the dataset to parse the features
    images = dataset.map(parseFunction)
    
    # Directory to save the extracted images
    output_dir = 'extracted_images'
    os.makedirs(output_dir, exist_ok=True)
                
    # Iterate over each example
    for i, (image_tensor, label_tensor, text_tensor) in enumerate(images):
        # Save the image
        image_path = os.path.join(output_dir, f'image_{i}.jpg')
        # tf.io.write_file(image_path, tf.image.encode_jpeg(image_tensor))
        print(f'Saved {image_path}')
        print(f'Label: {label_tensor.numpy()}, Text: {text_tensor.numpy()}')
save_images(valid_dataset)

Saved extracted_images/image_0.jpg
Label: 1.0, Text: [b'enemyHead']
Saved extracted_images/image_1.jpg
Label: 1.0, Text: [b'enemyHead' b'enemyBody']
Saved extracted_images/image_2.jpg
Label: 1.0, Text: [b'enemyHead' b'enemyBody']
Saved extracted_images/image_3.jpg
Label: 1.0, Text: [b'enemyHead']
Saved extracted_images/image_4.jpg
Label: 1.0, Text: [b'enemyHead' b'enemyBody']
Saved extracted_images/image_5.jpg
Label: 1.0, Text: [b'enemyHead' b'enemyBody']
Saved extracted_images/image_6.jpg
Label: 1.0, Text: [b'enemyHead' b'enemyBody']
Saved extracted_images/image_7.jpg
Label: 1.0, Text: [b'enemyHead' b'enemyBody']
Saved extracted_images/image_8.jpg
Label: 1.0, Text: [b'enemyHead' b'enemyBody']
Saved extracted_images/image_9.jpg
Label: 1.0, Text: [b'enemyHead' b'enemyBody']
Saved extracted_images/image_10.jpg
Label: 1.0, Text: [b'enemyHead' b'enemyBody']
Saved extracted_images/image_11.jpg
Label: 1.0, Text: [b'enemyHead' b'enemyBody']
Saved extracted_images/image_12.jpg
Label: 1.0, Text

2024-04-15 12:51:44.001942: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [22]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(416, 416, 3)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    tf.keras.layers.Flatten(),
    # tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_1 (Conv2D)               │ (None, 416, 416, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 208, 208, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 2768896)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 2768896)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │     2,768,897 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,770,689 (10.57 MB)

 Trainable params: 2,770,689 (10.57 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
batched_train = train_dataset.map(_parse_function).shuffle(1024).batch(32)
batched_valid = valid_dataset.map(_parse_function).shuffle(1024).batch(32)

history = model.fit(batched_train, epochs=10)

Epoch 1/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 56s 3s/step - accuracy: 0.8386 - loss: 10.6397
Epoch 2/10


2024-04-14 21:30:05.608853: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/Users/max/anaconda3/envs/CS6476Project/lib/python3.11/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


14/14 ━━━━━━━━━━━━━━━━━━━━ 48s 3s/step - accuracy: 0.8744 - loss: 2.0065
Epoch 3/10


2024-04-14 21:30:53.341946: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


14/14 ━━━━━━━━━━━━━━━━━━━━ 47s 3s/step - accuracy: 0.8997 - loss: 0.2710
Epoch 4/10


...

2024-04-14 21:35:33.940166: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


14/14 ━━━━━━━━━━━━━━━━━━━━ 47s 3s/step - accuracy: 1.0000 - loss: 0.0101
Epoch 10/10


2024-04-14 21:36:20.930306: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


14/14 ━━━━━━━━━━━━━━━━━━━━ 46s 3s/step - accuracy: 1.0000 - loss: 0.0041


2024-04-14 21:37:07.377406: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


In [8]:
train_accuracy = model.evaluate(batched_train)
print("Training Accuracy:", train_accuracy)

valid_accuracy = model.evaluate(batched_valid)
print("Validation Accuracy:", valid_accuracy)


14/14 ━━━━━━━━━━━━━━━━━━━━ 13s 863ms/step - accuracy: 1.0000 - loss: 0.0076
Training Accuracy: [0.005816842429339886, 1.0]


2024-04-14 21:37:20.019786: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 578ms/step - accuracy: 0.9145 - loss: 0.2902
Validation Accuracy: [0.15749137103557587, 0.9305555820465088]


2024-04-14 21:37:22.282637: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


In [24]:
# Pre-trained network
from tensorflow.keras.applications import MobileNetV2

pretrained_model = MobileNetV2((416, 416, 3), 0.35, include_top=False, weights='imagenet')


model = tf.keras.models.Sequential([
    pretrained_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

/var/folders/xl/gsbth_zx5kz5wb2wm4n5p9s40000gn/T/ipykernel_22635/1738588732.py:3: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  pretrained_model = MobileNetV2((416, 416, 3), 0.35, include_top=False, weights='imagenet')


2019640/2019640 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [25]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_0.35_224            │ ?                      │       410,208 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_3      │ ?                      │   0 (unbuilt) │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 410,208 (1.56 MB)

 Trainable params: 396,128 (1.51 MB)

 Non-trainable params: 14,080 (55.00 KB)

In [26]:
batched_train = train_dataset.map(_parse_function).shuffle(1024).batch(32)
batched_valid = valid_dataset.map(_parse_function).shuffle(1024).batch(32)

history = model.fit(batched_train, epochs=10)

Epoch 1/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 154s 8s/step - accuracy: 0.6808 - loss: 0.5636
Epoch 2/10


2024-04-15 12:56:41.802669: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


14/14 ━━━━━━━━━━━━━━━━━━━━ 123s 9s/step - accuracy: 0.9904 - loss: 0.0626
Epoch 3/10


2024-04-15 12:58:45.341386: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


14/14 ━━━━━━━━━━━━━━━━━━━━ 127s 9s/step - accuracy: 0.9981 - loss: 0.0136
Epoch 4/10


...

2024-04-15 13:11:16.348457: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


14/14 ━━━━━━━━━━━━━━━━━━━━ 130s 9s/step - accuracy: 1.0000 - loss: 4.0838e-04
Epoch 10/10


2024-04-15 13:13:26.800548: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


14/14 ━━━━━━━━━━━━━━━━━━━━ 119s 8s/step - accuracy: 1.0000 - loss: 5.5633e-04


2024-04-15 13:15:26.015940: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


In [27]:
train_accuracy = model.evaluate(batched_train)
print("Training Accuracy:", train_accuracy)

valid_accuracy = model.evaluate(batched_valid)
print("Validation Accuracy:", valid_accuracy)

14/14 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - accuracy: 0.9149 - loss: 0.9771
Training Accuracy: [0.7788737416267395, 0.9265402555465698]


2024-04-15 13:15:48.324899: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 989ms/step - accuracy: 0.9184 - loss: 1.5184
Validation Accuracy: [1.3535583019256592, 0.9305555820465088]


2024-04-15 13:15:52.098296: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
